In [1]:
import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os, copy

import gc

# Gym is an OpenAI toolkit for RL
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

# NES Emulator for OpenAI Gym
from nes_py.wrappers import JoypadSpace

# Super Mario environment for OpenAI Gym
import gym_super_mario_bros

In [2]:
# Empty cached memory using the cached allocator
torch.backends.cudnn.benchmark = True
torch.cuda.empty_cache()

In [3]:
env = gym_super_mario_bros.make("SuperMarioBros-1-1-v3", render_mode='rgb', apply_api_compatibility=True)

/opt/parallax/env38/lib/python3.8/site-packages/gym/envs/registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


In [4]:
''''
from IPython.display import HTML
from IPython import display as ipythondisplay
import glob
import io
import base64

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data=''''''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
'''

'\'\nfrom IPython.display import HTML\nfrom IPython import display as ipythondisplay\nimport glob\nimport io\nimport base64\n\nfrom pyvirtualdisplay import Display\ndisplay = Display(visible=0, size=(1400, 900))\ndisplay.start()\n\n"""\nUtility functions to enable video recording of gym environment and displaying it\nTo enable video, just do "env = wrap_env(env)""\n"""\n\ndef show_video():\n  mp4list = glob.glob(\'video/*.mp4\')\n  if len(mp4list) > 0:\n    mp4 = mp4list[0]\n    video = io.open(mp4, \'r+b\').read()\n    encoded = base64.b64encode(video)\n    ipythondisplay.display(HTML(data=.format(encoded.decode(\'ascii\'))))\n  else: \n    print("Could not find video")\n'

In [5]:
#   Limit the action-space to 
# 0. walk right
# 1. jump right

env = JoypadSpace(env, [["right"], ["right", "A"]])

env.reset()
next_state, reward, done, trunc, info = env.step(action = 0)
print(f'{next_state.shape},\n {reward},\n {done},\n {info}')


(240, 256, 3),
 0,
 False,
 {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'y_pos': 79}


In [6]:
if torch.cuda.is_available():
    print(f'Cuda device available')

Cuda device available


In [7]:
class SkipFrame(gym.Wrapper):
    
    def __init__(self, env, skip):
        super().__init__(env)
        self._skip = skip
        
    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        for i in range(self._skip):
            #    Accumulate reward and repeat the same action
            obs, reward, done, trunk, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, trunk, info
    
class GrayScaleObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape[:2]
        self.observation_space = Box(low = 0, high = 255, shape = obs_shape, dtype=np.uint8)
        
    def permute_orientation(self, observation):
        # Permute [H,W,C] array to [C, H, W] tensor
        observation = np.transpose(observation, (2, 0, 1))
        observation = torch.tensor(observation.copy(), dtype = torch.float)
        return observation
    
    def observation(self, observation):
        observation = self.permute_orientation(observation)
        transform = T.Grayscale()
        observation = transform(observation)
        return observation
    
class ResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        if isinstance(shape, int):
            self.shape = (shape, shape)
        else:
            self.shape = tuple(shape)
        
        obs_shape = self.shape + self.observation_space.shape[2:]
        self.observation_space = Box(low = 0, high = 255, shape=obs_shape, dtype = np.uint8)
        
    def observation(self, observation):
        transforms = T.Compose(
            [T.Resize(self.shape), T.Normalize(0,255)]
        )
        observation = transforms(observation).squeeze(0)
        return observation

In [8]:
#    Apply Wrappers to Environment
env = SkipFrame(env, skip = 4)
env = GrayScaleObservation(env)
env = ResizeObservation(env, shape = 84)
if gym.__version__ < '0.26':
    env = FrameStack(env, num_stack=4, new_step_api=True)
else:
    env = FrameStack(env, num_stack=4)

In [9]:
class Mario:
    def __init__():
        pass

    def act(self, state):
        """Given a state, choose an epsilon-greedy action"""
        pass

    def cache(self, experience):
        """Add the experience to memory"""
        pass

    def recall(self):
        """Sample experiences from memory"""
        pass

    def learn(self):
        """Update online action value (Q) function with a batch of experiences"""
        pass

# Act

An action consists of exploit and explore

When exploit is used, the MarioNet is used to provide the optimal action

In [10]:
class Mario:
    
    def __init__(self, state_dim, action_dim, save_dir):
        
        self.state_dim  = state_dim
        self.action_dim = action_dim
        self.save_dir   = save_dir
        self.use_cuda   = torch.cuda.is_available()
        self.device = "cuda" if self.use_cuda else "cpu"
        
        # Mario's DNN to predict the most optimal action - we implement this in the Learn section
        self.net = MarioNet(self.state_dim, self.action_dim).float()
        self.net = self.net.to(device = self.device)
        
        self.exploration_rate = 1.0
        self.exploration_rate_decay = 0.99999975
        self.exploration_rate_min = 0.1
        self.curr_step = 0
        
        self.save_every = 5e5 # Number of experiences before saving Mario Net
        
    def act(self, state):
        """
        Given a state, choose an epsilon-greedy action and update value of step.

        Inputs:
        state(LazyFrame): A single observation of the current state, dimension is (state_dim)
        Outputs:
        action_idx (int): An integer representing which action Mario will perform
        """
        
        #    Explore
        if np.random.rand() < self.exploration_rate:
            action_idx = np.random.randint(self.action_dim)
        
        
        #    Exploit
        else:
            state = state[0].__array__() if isinstance(state, tuple) else state.__array__()
            state = torch.tensor(state, device = self.device).unsqueeze(0)
            action_values = self.net(state, model = "online")
            action_idx = torch.argmax(action_values, axis = 1).item()
            
        # decrease exploration rate
        self.exploration_rate *= self.exploration_rate_decay
        self.exploration_date  = max(self.exploration_rate_min, self.exploration_rate)
        
        # increment step
        self.curr_step += 1
        return action_idx

# Cache and Recall

These two methods function as marios "memory"

## cache
For each action, the experience is stores in memory. The experience includes: current state, action performed, reward for the action, the next state and is the game is done

## recall
Randomly sample a batch of experiences from Marios memory to learn the game 



In [11]:
class Mario(Mario):

    def __init__(self, state_dim, action_dim, save_dir):
        super().__init__(state_dim, action_dim, save_dir)
        self.memory = deque(maxlen=20000)
        self.batch_size = 32
        
    def cache(self, state, next_state, action, reward, done):
        """
        Store the experience to self.memory (replay buffer)

        Inputs:
        state (LazyFrame),
        next_state (LazyFrame),
        action (int),
        reward (float),
        done(bool))
        """
        def first_if_tuple(x):
            return x[0] if isinstance(x, tuple) else x
        state = first_if_tuple(state).__array__()
        next_state = first_if_tuple(next_state).__array__()
        
        state = torch.FloatTensor(state)
        next_state = torch.FloatTensor(next_state)
        action = torch.LongTensor([action])
        reward = torch.DoubleTensor([reward])
        done   = torch.BoolTensor([done]).cuda()
    
        self.memory.append((state, next_state, action, reward, done,))
        
    def recall(self):
        """
        Retrieve a batch of memories from memory
        """
        batch = random.sample(self.memory, self.batch_size)
        state, next_state, action, reward, done = map(torch.stack, zip(*batch))
        
        if self.use_cuda:
            state, next_state, action, reward, done = state.cuda(), next_state.cuda(), action.cuda(), reward.cuda(), done.cuda()
        
        return state, next_state, action.squeeze(), reward.squeeze(), done.squeeze()

# Learn

DDQN algorithmn
[q_learning](chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://arxiv.org/pdf/1509.06461.pdf)

Uses two networks Qonline and Qtarget that independently approximate optimal action-value 

We share a feature generator `features` across Qonline and Qtarget

In [12]:
class MarioNet(nn.Module):
    """
    input -> (conv+relu)*3 -> flatten -> (dense + relu) X 2 -> output
    """
    def __init__(self, input_dim, output_dim):
        super().__init__()
        c, h, w = input_dim

        if h != 84:
            raise ValueError(f"Expecting input height: 84, got: {h}")
        if w != 84:
            raise ValueError(f"Expecting input width: 84, got: {w}")
            
        self.online = nn.Sequential(
            nn.Conv2d(in_channels = c, out_channels = 32, kernel_size = 8, stride = 4),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )
        
        self.target = copy.deepcopy(self.online)
        
        # Q_target params are frozen
        for p in self.target.parameters():
            p.requires_grad = False
            
    def forward(self, input, model):
        if model == "online":
            return self.online(input)
        elif model == "target":
            return self.target(input)

# TD Estimate and Target

TD Estimate - the predicted Q* for a given state 

TD Target - Aggregation of current reward and estimated Q* in the next state s'

@torch.no_grad() halts backpropagation on target

In [13]:
class Mario(Mario):
    def __init__(self, state_dim, action_dim, save_dir):
        super().__init__(state_dim, action_dim, save_dir)
        self.gamma = 0.9

    def td_estimate(self, state, action):
        '''
        current_Q = self.net(state, model="online")[
            np.arange(0, self.batch_size), action
        ].astype(float)  # Q_online(s,a)
        '''
        curr_state_q = self.net(state, model='online')
        current_Q =  curr_state_q[np.arange(0, self.batch_size), action]
        return current_Q

    @torch.no_grad()
    def td_target(self, reward, next_state, done):
        next_state_Q = self.net(next_state, model="target")
        best_action = torch.argmax(next_state_Q, axis=1)
        '''
        next_Q = self.net(next_state, model="target")[
            np.arange(0, self.batch_size), best_action
        ]
        '''
        next_state_q = self.net(next_state, model = 'target')
        next_Q = next_state_q[np.arange(0, self.batch_size), action]
        return (reward + (1 - done.float()) * self.gamma * next_Q).float()

In [14]:
class Mario(Mario):
    def __init__(self, state_dim, action_dim, save_dir):
        super().__init__(state_dim, action_dim, save_dir)
        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=0.00025)
        self.loss_fn = torch.nn.SmoothL1Loss()

    def update_Q_online(self, td_estimate, td_target):
        loss = self.loss_fn(td_estimate, td_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

    def sync_Q_target(self):
        self.net.target.load_state_dict(self.net.online.state_dict())

In [15]:
class Mario(Mario):
    def save(self):
        save_path = (
            self.save_dir / f"mario_net_{int(self.curr_step // self.save_every)}.chkpt"
        )
        torch.save(
            dict(model=self.net.state_dict(), exploration_rate=self.exploration_rate),
            save_path,
        )
        print(f"MarioNet saved to {save_path} at step {self.curr_step}")

In [16]:
class Mario(Mario):
    def __init__(self, state_dim, action_dim, save_dir):
        super().__init__(state_dim, action_dim, save_dir)
        self.burnin = 1e4  # min. experiences before training
        self.learn_every = 3  # no. of experiences between updates to Q_online
        self.sync_every = 1e4  # no. of experiences between Q_target & Q_online sync

    def learn(self):
        if self.curr_step % self.sync_every == 0:
            self.sync_Q_target()

        if self.curr_step % self.save_every == 0:
            self.save()

        if self.curr_step < self.burnin:
            return None, None

        if self.curr_step % self.learn_every != 0:
            return None, None

        # Sample from memory
        state, next_state, action, reward, done = self.recall()

        # Get TD Estimate
        td_est = self.td_estimate(state, action)

        # Get TD Target
        td_tgt = self.td_target(reward, next_state, done)

        # Backpropagate loss through Q_online
        loss = self.update_Q_online(td_est, td_tgt)

        return (td_est.mean().item(), loss)

In [17]:
import numpy as np
import time, datetime
import matplotlib.pyplot as plt


class MetricLogger:
    def __init__(self, save_dir):
        self.save_log = save_dir / "log"
        with open(self.save_log, "w") as f:
            f.write(
                f"{'Episode':>8}{'Step':>8}{'Epsilon':>10}{'MeanReward':>15}"
                f"{'MeanLength':>15}{'MeanLoss':>15}{'MeanQValue':>15}"
                f"{'TimeDelta':>15}{'Time':>20}\n"
            )
        self.ep_rewards_plot = save_dir / "reward_plot.jpg"
        self.ep_lengths_plot = save_dir / "length_plot.jpg"
        self.ep_avg_losses_plot = save_dir / "loss_plot.jpg"
        self.ep_avg_qs_plot = save_dir / "q_plot.jpg"

        # History metrics
        self.ep_rewards = []
        self.ep_lengths = []
        self.ep_avg_losses = []
        self.ep_avg_qs = []

        # Moving averages, added for every call to record()
        self.moving_avg_ep_rewards = []
        self.moving_avg_ep_lengths = []
        self.moving_avg_ep_avg_losses = []
        self.moving_avg_ep_avg_qs = []

        # Current episode metric
        self.init_episode()

        # Timing
        self.record_time = time.time()

    def log_step(self, reward, loss, q):
        self.curr_ep_reward += reward
        self.curr_ep_length += 1
        if loss:
            self.curr_ep_loss += loss
            self.curr_ep_q += q
            self.curr_ep_loss_length += 1

    def log_episode(self):
        "Mark end of episode"
        self.ep_rewards.append(self.curr_ep_reward)
        self.ep_lengths.append(self.curr_ep_length)
        if self.curr_ep_loss_length == 0:
            ep_avg_loss = 0
            ep_avg_q = 0
        else:
            ep_avg_loss = np.round(self.curr_ep_loss / self.curr_ep_loss_length, 5)
            ep_avg_q = np.round(self.curr_ep_q / self.curr_ep_loss_length, 5)
        self.ep_avg_losses.append(ep_avg_loss)
        self.ep_avg_qs.append(ep_avg_q)

        self.init_episode()

    def init_episode(self):
        self.curr_ep_reward = 0.0
        self.curr_ep_length = 0
        self.curr_ep_loss = 0.0
        self.curr_ep_q = 0.0
        self.curr_ep_loss_length = 0

    def record(self, episode, epsilon, step):
        mean_ep_reward = np.round(np.mean(self.ep_rewards[-100:]), 3)
        mean_ep_length = np.round(np.mean(self.ep_lengths[-100:]), 3)
        mean_ep_loss = np.round(np.mean(self.ep_avg_losses[-100:]), 3)
        mean_ep_q = np.round(np.mean(self.ep_avg_qs[-100:]), 3)
        self.moving_avg_ep_rewards.append(mean_ep_reward)
        self.moving_avg_ep_lengths.append(mean_ep_length)
        self.moving_avg_ep_avg_losses.append(mean_ep_loss)
        self.moving_avg_ep_avg_qs.append(mean_ep_q)

        last_record_time = self.record_time
        self.record_time = time.time()
        time_since_last_record = np.round(self.record_time - last_record_time, 3)

        print(
            f"Episode {episode} - "
            f"Step {step} - "
            f"Epsilon {epsilon} - "
            f"Mean Reward {mean_ep_reward} - "
            f"Mean Length {mean_ep_length} - "
            f"Mean Loss {mean_ep_loss} - "
            f"Mean Q Value {mean_ep_q} - "
            f"Time Delta {time_since_last_record} - "
            f"Time {datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}"
        )

        with open(self.save_log, "a") as f:
            f.write(
                f"{episode:8d}{step:8d}{epsilon:10.3f}"
                f"{mean_ep_reward:15.3f}{mean_ep_length:15.3f}{mean_ep_loss:15.3f}{mean_ep_q:15.3f}"
                f"{time_since_last_record:15.3f}"
                f"{datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S'):>20}\n"
            )

        for metric in ["ep_rewards", "ep_lengths", "ep_avg_losses", "ep_avg_qs"]:
            plt.plot(getattr(self, f"moving_avg_{metric}"))
            plt.savefig(getattr(self, f"{metric}_plot"))
            plt.clf()

In [ ]:
use_cuda = torch.cuda.is_available()
print(f"Using CUDA: {use_cuda}")
print()

save_dir = Path("checkpoints") / datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
save_dir.mkdir(parents=True)

mario = Mario(state_dim=(4, 84, 84), action_dim=env.action_space.n, save_dir=save_dir)

logger = MetricLogger(save_dir)
episodes = 10000
for e in range(episodes):
    state = env.reset()

    # Play the game!
    while True:

        # Run agent on the state
        action = mario.act(state)

        # Agent performs action
        next_state, reward, done, trunc, info = env.step(action)

        # Remember
        mario.cache(state, next_state, action, reward, done)

        # Learn
        q, loss = mario.learn()
        
        # Logging
        logger.log_step(reward, loss, q)

        #    Clear non essential variables from GPU memory
        del q, loss
        
        # Update state
        state = next_state

        # Check if end of game
        if done or info["flag_get"]:
            break

    logger.log_episode()

    if e % 20 == 0:
        logger.record(episode=e, epsilon=mario.exploration_rate, step=mario.curr_step)

Using CUDA: True

Episode 0 - Step 48 - Epsilon 0.9999880000704982 - Mean Reward 232.0 - Mean Length 48.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.47 - Time 2023-01-16T19:26:19
Episode 20 - Step 7208 - Epsilon 0.9981996224020983 - Mean Reward 720.333 - Mean Length 343.238 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 72.537 - Time 2023-01-16T19:27:32
Episode 40 - Step 13775 - Epsilon 0.9965621724766063 - Mean Reward 672.829 - Mean Length 335.976 - Mean Loss 0.12 - Mean Q Value 0.406 - Time Delta 99.44 - Time 2023-01-16T19:29:11
Episode 60 - Step 21620 - Epsilon 0.9946095800583177 - Mean Reward 700.377 - Mean Length 354.426 - Mean Loss 0.18 - Mean Q Value 1.059 - Time Delta 255.952 - Time 2023-01-16T19:33:27
Episode 80 - Step 27386 - Epsilon 0.9931768830338418 - Mean Reward 685.074 - Mean Length 338.099 - Mean Loss 0.206 - Mean Q Value 1.735 - Time Delta 211.903 - Time 2023-01-16T19:36:59
Episode 100 - Step 34587 - Epsilon 0.9913905245549917 - Mean Reward 705.59 - Mean Length

/tmp/ipykernel_31648/2888092854.py:28: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  done   = torch.BoolTensor([done]).cuda()


Episode 180 - Step 65302 - Epsilon 0.9838070367313226 - Mean Reward 766.51 - Mean Length 379.16 - Mean Loss 0.335 - Mean Q Value 7.001 - Time Delta 142.725 - Time 2023-01-16T19:49:42
Episode 200 - Step 72130 - Epsilon 0.9821291104271858 - Mean Reward 763.47 - Mean Length 375.43 - Mean Loss 0.361 - Mean Q Value 7.967 - Time Delta 117.009 - Time 2023-01-16T19:51:39
Episode 220 - Step 77936 - Epsilon 0.9807045839453351 - Mean Reward 730.68 - Mean Length 353.43 - Mean Loss 0.398 - Mean Q Value 9.032 - Time Delta 97.766 - Time 2023-01-16T19:53:17
Episode 240 - Step 84180 - Epsilon 0.9791748981287696 - Mean Reward 710.97 - Mean Length 342.09 - Mean Loss 0.439 - Mean Q Value 10.054 - Time Delta 106.653 - Time 2023-01-16T19:55:03
Episode 260 - Step 91569 - Epsilon 0.9773677866792466 - Mean Reward 701.8 - Mean Length 342.11 - Mean Loss 0.46 - Mean Q Value 10.877 - Time Delta 127.841 - Time 2023-01-16T19:57:11
Episode 280 - Step 98585 - Epsilon 0.9756549859324496 - Mean Reward 677.14 - Mean Leng